In [ ]:
import torch
import numpy as np
import pandas as pd
from glob import glob
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score

In [ ]:
train_paths = glob('../input/dstc-inference/inference/*_dev_logit_truth.pt')
valid_paths = glob('../input/dstc-inference/inference/*_devtest_logit_truth.pt')
train_paths, valid_paths

In [ ]:
def read_one_model(train_path, valid_path):
    train = torch.load(train_path, map_location=torch.device('cpu'))
    X_train = train['logit'].squeeze().numpy()
    y_train = train['truth'].squeeze().numpy()
    valid = torch.load(valid_path, map_location=torch.device('cpu'))
    X_valid = valid['logit'].squeeze().numpy()
    y_valid = valid['truth'].squeeze().numpy()
    return X_train, y_train, X_valid, y_valid

X_train, y_train, X_valid, y_valid = [], [], [], []
for i, tp in enumerate(train_paths):
    vp = [x for x in valid_paths if tp.split('/')[-1].split('_')[0] in x]
    assert len(vp) == 1
    vp = vp[0]
    out1, out2, out3, out4 = read_one_model(tp, vp)
    X_train.append(out1)
    y_train.append(out2)
    X_valid.append(out3)
    y_valid.append(out4)

# order check
for i in range(1, len(y_train)):
    assert all(y_train[i] == y_train[0])
    assert all(y_valid[i] == y_valid[0])
X_train = np.array(X_train).transpose(1,0)
y_train = np.array(y_train[0])
X_valid = np.array(X_valid).transpose(1,0)
y_valid = np.array(y_valid[0])

In [ ]:
for i in range(0, X_train.shape[1]):
    print(f1_score(y_train, (X_train[:, i] > 0).astype(int)))
    print(f1_score(y_valid, (X_valid[:, i] > 0).astype(int)))

In [ ]:
weight = [1,1,1]
X_valid_blend = np.average(X_valid, weights=weight, axis=1)
X_valid_blend.shape, f1_score(y_valid, (X_valid_blend > 0).astype(int))

In [ ]:
def objective(trial):
    w1 = trial.suggest_uniform('w1', 0, 1)
    w2 = trial.suggest_uniform('w2', 0, 1)
    w3 = 1-w1-w2
    X_train_blend = np.average(X_train, weights=[w1,w2,w3], axis=1)
    return f1_score(y_train, (X_train_blend > 0).astype(int))
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

In [ ]:
weights = {'w1': 0.3810618627395184, 'w2': 0.2285650147095583}
w3 = 1-weights['w1']-weights['w2']
w3

In [ ]:
weight = [weights['w1'], weights['w2'], w3]
X_valid_blend = np.average(X_valid, weights=weight, axis=1)
X_valid_blend.shape, f1_score(y_valid, (X_valid_blend > 0).astype(int))